In [1]:
import pandas as pd
import numpy as np
from lenstronomy.Data.psf import PSF
import autolens as al
import os
from astropy.io import fits
import utils

## Generate a autolens + imfit run

In [2]:
modelized_systems = pd.read_csv('test_dataset.csv')
modelized_systems.head()

,Unnamed: 0,OBJID-g,H0-g,Om0-g,exposure_time-g,numPix-g,pixel_scale-g,psf_type-g,read_noise-g,ccd_gain-g,...,PLANE_1-REDSHIFT-g,PLANE_2-REDSHIFT-g,PLANE_1-REDSHIFT-r,PLANE_2-REDSHIFT-r,PLANE_1-REDSHIFT-i,PLANE_2-REDSHIFT-i,PLANE_1-REDSHIFT-z,PLANE_2-REDSHIFT-z,PLANE_1-REDSHIFT-Y,PLANE_2-REDSHIFT-Y
0,1501,1502,73.0,0.3,90.0,100,0.263,GAUSSIAN,7,6.083,...,0.520542,1.401964,0.520542,1.401964,0.520542,1.401964,0.520542,1.401964,0.520542,1.401964
1,2586,2587,73.0,0.3,90.0,100,0.263,GAUSSIAN,7,6.083,...,0.432048,2.126668,0.432048,2.126668,0.432048,2.126668,0.432048,2.126668,0.432048,2.126668
2,2653,2654,73.0,0.3,90.0,100,0.263,GAUSSIAN,7,6.083,...,0.870553,3.289295,0.870553,3.289295,0.870553,3.289295,0.870553,3.289295,0.870553,3.289295
3,1055,1056,73.0,0.3,90.0,100,0.263,GAUSSIAN,7,6.083,...,0.856923,2.497602,0.856923,2.497602,0.856923,2.497602,0.856923,2.497602,0.856923,2.497602
4,705,706,73.0,0.3,90.0,100,0.263,GAUSSIAN,7,6.083,...,0.566559,3.057067,0.566559,3.057067,0.566559,3.057067,0.566559,3.057067,0.566559,3.057067


In [3]:
file = open("runner.sh", "w") 
for i in range(0, 50):
    file.write('python3 Pipeline.py '+ str(int(modelized_systems['OBJID-g'][i]))+'\n')
file.close() 

## Generate a galfit run

In [4]:
for i in range(0, 50):
    
    cutout = fits.open('./simulations/fits_files/i/'+str(modelized_systems['OBJID-g'][i]-1)+'.fits')[0].data

    teste = utils.lens_light_test(cutout)
    has_lens_light, f, x, y = teste.test()
    
    if has_lens_light:
        
        
        pixel_scale = float(modelized_systems['pixel_scale-g'][i])
        seeing = float(modelized_systems['seeing-i'][i])
        sky_rms = float(modelized_systems['sky_brightness-i'][i])
        expo_time = float(modelized_systems['exposure_time-i'][i])
        zl = float(modelized_systems['PLANE_1-REDSHIFT-g'][i])
        zs = float(modelized_systems['PLANE_2-REDSHIFT-g'][i])
        ccd_gain = float(modelized_systems['ccd_gain-i'][i])
        read_noise = float(modelized_systems['read_noise-i'][i])
        zero_point = float(modelized_systems['magnitude_zero_point-i'][i])

        kwargs_psf = {'psf_type': 'GAUSSIAN',
                      'fwhm': seeing,
                      'pixel_size': pixel_scale,
                      'truncation': 4/seeing}
        psf_class = PSF(**kwargs_psf)
        psf = psf_class.kernel_point_source/np.max(psf_class.kernel_point_source)
        os.mkdir('./galfit/'+str(modelized_systems['OBJID-g'][i]))
        # exporting psf
        hdu = fits.PrimaryHDU(data=psf)
        hdu.writeto('./galfit/'+str(modelized_systems['OBJID-g'][i])+'/psf.fits')
        # exporting mask
        
        finder = utils.find_radius(pre_set_sigma=3, image_array=cutout)
        _, _, mask_radius= finder.get_radius() # gaussian normalization, center position (1-d) and radius (sigma)

        radius_value = mask_radius*float(pixel_scale)
        #print(radius_value)
        if radius_value > 3.:
            radius_value = 3.
        #print(radius_value)

        mask = 1-al.Mask2D.circular(shape_native=(100, 100), pixel_scales=pixel_scale, radius=radius_value)
        mask.output_to_fits('./galfit/'+str(modelized_systems['OBJID-g'][i])+'/mask.fits')
        # exporting feedme file
        f = open('./galfit/'+str(modelized_systems['OBJID-g'][i])+'/galfit.feedme', 'a') 
        f.write('\n')
        f.write('=============================================================================== \n')
        f.write(
                'A) /home/joao/AutoLensFit/simulations/fits_files/i/'+str(modelized_systems['OBJID-g'][i])+'.fits            # Input data image (FITS file) \n'+
                'B) ./'+str(modelized_systems['OBJID-g'][i])+'/'+'galfit_image.fits         # Output data image block \n'+
                'C) none                # Sigma image name (made from data if blank or "none") \n'+
                'D) psf.fits   #        # Input PSF image and (optional) diffusion kernel \n'+
                'E) 1                   # PSF fine sampling factor relative to data  \n'+
                'F) mask.fits                # Bad pixel mask (FITS image or ASCII coord list) \n'+
                'G) none                # File with parameter constraints (ASCII file)  \n'+
                'H) 1    100   1    100   # Image region to fit (xmin xmax ymin ymax) \n'+
                'I) 100    100          # Size of the convolution box (x y) \n'+
                'J) '+str(zero_point)+' # Magnitude photometric zeropoint  \n'+
                'K) 0.038  0.038        # Plate scale (dx dy)    [arcsec per pixel] \n'+
                'O) regular             # Display type (regular, curses, both) \n'+
                'P) 0                   # Choose: 0=optimize, 1=model, 2=imgblock, 3=subcomps \n')
        f.write('\n')
        f.write(
                '# INITIAL FITTING PARAMETERS \n'+
                '# \n'+
                '#   For object type, the allowed functions are: \n'+
                '#       nuker, sersic, expdisk, devauc, king, psf, gaussian, moffat, \n'+
                '#       ferrer, powsersic, sky, and isophote. \n'+
                '# \n'+
                '#   Hidden parameters will only appear when theyre specified: \n'+
                '#       C0 (diskyness/boxyness),  \n'+
                '#       Fn (n=integer, Azimuthal Fourier Modes), \n'+
                '#       R0-R10 (PA rotation, for creating spiral structures). \n'+
                '# \n'+ 
                '# -----------------------------------------------------------------------------\n'+
                '#   par)    par value(s)    fit toggle(s)    # parameter description \n'+
                '# -----------------------------------------------------------------------------\n'+
                '# Object number: 1 \n'+
                '0) sersic                 #  object type \n'+
                '1) 50.  50.  1 1  #  position x, y \n'+
                '3) 20.0890     1          #  Integrated magnitude \n'+
                '4) 8.4      1          #  R_e (half-light radius)   [pix] \n'+
                '5) 2.3      1          #  Sersic index n (de Vaucouleurs n=4) \n'+
                '6) 0.0000      0          #     ----- \n'+
                '7) 0.0000      0          #     ----- \n'+
                '8) 0.0000      0          #     ----- \n'+
                '9) 1.      0          #  axis ratio (b/a) \n'+
                '10) 90.0    1          #  position angle (PA) [deg: Up=0, Left=90] \n'+
                'Z) 0                      #  output option (0 = resid., 1 = Dont subtract) \n'
                )
        f.write('\n')
        f.write(
                '# Object number: 2 \n'+
                '0) sky                    #  object type \n'+
                '1) '+str(np.median(cutout))+'      1          #  sky background at center of fitting region [ADUs] \n'+
                '2) 0.0000      0          #  dsky/dx (sky gradient in x) \n'+
                '3) 0.0000      0          #  dsky/dy (sky gradient in y) \n'+
                'Z) 0                      #  output option (0 = resid., 1 = Dont subtract) \n'
                )
        f.close()

In [5]:
file = open("./galfit/galfit_runner.sh", "w") 
#file.write('#!/bin/bash \n')
for i in range(0, 50):
    cutout = fits.open('./simulations/fits_files/i/'+str(modelized_systems['OBJID-g'][i]-1)+'.fits')[0].data
    teste = utils.lens_light_test(cutout)
    has_lens_light, f, x, y = teste.test()
    #file.write('galfit')
    if has_lens_light:
        file.write('/home/joao/programs/galfit/galfit '+'./'+str(modelized_systems['OBJID-g'][i])+'/galfit.feedme \n')
        file.write('mv ./'+str(modelized_systems['OBJID-g'][i])+'/galfit_image.fits /home/joao/AutoLensFit/fits_results/lens_light_sphsersic/'+str(modelized_systems['OBJID-g'][i])+'_Galfit[SPHSERSIC].fits \n')
file.close() 